In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('Molecule CogsSGAMarketing formulas(Dummy Data).csv') 
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,2025,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,2020,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Company Name,Molecule Name,Therapuetic Area,Indication,Company Revenue,Company Cost of goods sold,Company SGA,Company Marketing,Molecule Revenue,NaN,...,Company SGA,Company Marketing,Molecule Revenue,NaN,Company Revenue,Company Cost of goods sold,Company SGA,Company Marketing,Molecule Revenue,NaN
1,Apex Therapeutics,Molecule_005,Infectious Diseases,HIV/AIDS,"$14,248,000,000","$5,000,400,000","$2,849,600,000","$1,709,760,000","$113,984,000",NaN,...,"$2,673,090,000","$1,527,480,000","$190,935,000",NaN,"$8,460,000,000","$3,891,600,000","$1,522,800,000","$1,015,200,000","$126,900,000",NaN
2,Apex Therapeutics,Molecule_025,Rare Diseases,Lysosomal Storage Disorders,"$14,248,000,000","$5,000,400,000","$2,849,600,000","$1,709,760,000","$142,480,000",NaN,...,"$2,673,090,000","$1,527,480,000","$190,935,000",NaN,"$8,460,000,000","$3,891,600,000","$1,522,800,000","$1,015,200,000","$126,900,000",NaN
3,Apex Therapeutics,Molecule_045,Neurology,Epilepsy,"$14,248,000,000","$5,000,400,000","$2,849,600,000","$1,709,760,000","$213,720,000",NaN,...,"$2,673,090,000","$1,527,480,000","$190,935,000",NaN,"$8,460,000,000","$3,891,600,000","$1,522,800,000","$1,015,200,000","$126,900,000",NaN
4,Apex Therapeutics,Molecule_065,Dermatology,Atopic Dermatitis,"$14,248,000,000","$5,000,400,000","$2,849,600,000","$1,709,760,000","$284,960,000",NaN,...,"$2,673,090,000","$1,527,480,000","$190,935,000",NaN,"$8,460,000,000","$3,891,600,000","$1,522,800,000","$1,015,200,000","$126,900,000",NaN


In [9]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [11]:
raw_df = pd.read_csv('Molecule CogsSGAMarketing formulas(Dummy Data).csv', header=None)

In [12]:
# Row 0 has years (2025, 2024, etc.)
years = raw_df.iloc[0].ffill().tolist()
# iloc - a label-based indexer used for integer-location based indexing of a DataFrame
# Row 1 has metric names
metrics = raw_df.iloc[1].tolist()


In [13]:
# Clean metrics: replace NaNs with empty string, then strip
# strip() removes all leading and trailing whitespace characters (spaces, tabs \t, newlines \n, carriage returns \r, etc.).
metrics_clean = []
for m in metrics:
    if pd.notnull(m):
        metrics_clean.append(str(m).strip())
    else:
        metrics_clean.append('')

new_cols = []

# zip() pairs up two lists element by element.
for year, metric in zip(years, metrics_clean):
    # Drop nan or empty prefix for the first 4 columns (Company Name, Molecule Name, etc.)
    if str(year).lower() == 'nan' or year == '':
        new_cols.append(metric)
    else:
        new_cols.append(f"{year}_{metric}")


In [15]:
# Assign clean column names to actual data (starting row 2)
df = pd.read_csv('Molecule CogsSGAMarketing formulas(Dummy Data).csv', header=2)
df.columns = new_cols
# Drop columns where the name ends with "_" (like '2025_')
df = df.loc[:, ~df.columns.str.endswith('_')]
print(df.columns)


Index(['Company Name', 'Molecule Name', 'Therapuetic Area', 'Indication',
       '2025_Company Revenue', '2025_Company Cost of goods sold',
       '2025_Company SGA', '2025_Company Marketing', '2025_Molecule Revenue',
       '2024_Company Revenue', '2024_Company Cost of goods sold',
       '2024_Company SGA', '2024_Company Marketing', '2024_Molecule Revenue',
       '2023_Company Revenue', '2023_Company Cost of goods sold',
       '2023_Company SGA', '2023_Company Marketing', '2023_Molecule Revenue',
       '2022_Company Revenue', '2022_Company Cost of goods sold',
       '2022_Company SGA', '2022_Company Marketing', '2022_Molecule Revenue',
       '2021_Company Revenue', '2021_Company Cost of goods sold',
       '2021_Company SGA', '2021_Company Marketing', '2021_Molecule Revenue',
       '2020_Company Revenue', '2020_Company Cost of goods sold',
       '2020_Company SGA', '2020_Company Marketing', '2020_Molecule Revenue'],
      dtype='object')


In [16]:
# drop rows where 'Comapny Name' is Nan or is the header string itself
df = df[df['Company Name'].notnull() & (df['Company Name'] != 'Company Name')]

In [17]:
# clean text fields
text_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area', 'Indication']
for col in text_cols:
  df[col] = df[col].str.strip()

In [18]:
# get all the financial columns (those with 'Revenue', 'Cost', 'SGA', 'Marketing' in name)
financial_cols = [col for col in df.columns if any(keyword in col for keyword in ['Revenue', 'Cost', 'SGA', 'Marketing'])]

# remove $, commas and convert to numeric
df[financial_cols] = df[financial_cols].replace({'\$': '', ',': ''}, regex=True)
df[financial_cols] = df[financial_cols].apply(pd.to_numeric, errors='coerce')

# When errors='coerce' is specified, any values that cannot be successfully converted to the target data type will be replaced with NaN (Not a Number).
# to_numeric - used to convert an argument to a numerical type


In [19]:
# Fill NaN in financial columns with 0
df[financial_cols] = df[financial_cols].fillna(0)
df.head()

,Company Name,Molecule Name,Therapuetic Area,Indication,2025_Company Revenue,2025_Company Cost of goods sold,2025_Company SGA,2025_Company Marketing,2025_Molecule Revenue,2024_Company Revenue,...,2021_Company Revenue,2021_Company Cost of goods sold,2021_Company SGA,2021_Company Marketing,2021_Molecule Revenue,2020_Company Revenue,2020_Company Cost of goods sold,2020_Company SGA,2020_Company Marketing,2020_Molecule Revenue
0,Apex Therapeutics,Molecule_025,Rare Diseases,Lysosomal Storage Disorders,14248000000,5000400000,2849600000,1709760000,142480000,8016000000,...,12729000000,4836000000,2673090000,1527480000,190935000,8460000000,3891600000,1522800000,1015200000,126900000
1,Apex Therapeutics,Molecule_045,Neurology,Epilepsy,14248000000,5000400000,2849600000,1709760000,213720000,8016000000,...,12729000000,4836000000,2673090000,1527480000,190935000,8460000000,3891600000,1522800000,1015200000,126900000
2,Apex Therapeutics,Molecule_065,Dermatology,Atopic Dermatitis,14248000000,5000400000,2849600000,1709760000,284960000,8016000000,...,12729000000,4836000000,2673090000,1527480000,190935000,8460000000,3891600000,1522800000,1015200000,126900000
3,Apex Therapeutics,Molecule_085,Oncology,Ovarian Cancer,14248000000,5000400000,2849600000,1709760000,213720000,8016000000,...,12729000000,4836000000,2673090000,1527480000,190935000,8460000000,3891600000,1522800000,1015200000,126900000
4,Apex Therapeutics,Molecule_105,Gastroenterology,Irritable Bowel Syndrome,14248000000,5000400000,2849600000,1709760000,284960000,8016000000,...,12729000000,4836000000,2673090000,1527480000,190935000,8460000000,3891600000,1522800000,1015200000,126900000


In [20]:
# get all the unique years from the column names
years = sorted(set(col.split('_')[0] for col in df.columns if col[:4].isdigit()))

In [21]:
#Step 1

for year in years:
  # column names
  company_rev_col = f"{year}_Company Revenue"
  molecule_rev_col = f"{year}_Molecule Revenue"
  ratio_col = f"{year}_Ratio"

  # calculate ratio (as percentage)
  df[ratio_col] = (df[molecule_rev_col] / df[company_rev_col]) * 100

  # find the position to insert the ratio column
  molecule_rev_idx = df.columns.get_loc(molecule_rev_col)
  cols = df.columns.tolist()

  # remove ratio_col from end and insert it at the correct position
  cols.remove(ratio_col)
  cols.insert(molecule_rev_idx + 1, ratio_col)

  # reassign columns in new order
  df = df[cols]

df.head()

,Company Name,Molecule Name,Therapuetic Area,Indication,2025_Company Revenue,2025_Company Cost of goods sold,2025_Company SGA,2025_Company Marketing,2025_Molecule Revenue,2025_Ratio,...,2021_Company SGA,2021_Company Marketing,2021_Molecule Revenue,2021_Ratio,2020_Company Revenue,2020_Company Cost of goods sold,2020_Company SGA,2020_Company Marketing,2020_Molecule Revenue,2020_Ratio
0,Apex Therapeutics,Molecule_025,Rare Diseases,Lysosomal Storage Disorders,14248000000,5000400000,2849600000,1709760000,142480000,1.000000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
1,Apex Therapeutics,Molecule_045,Neurology,Epilepsy,14248000000,5000400000,2849600000,1709760000,213720000,1.500000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
2,Apex Therapeutics,Molecule_065,Dermatology,Atopic Dermatitis,14248000000,5000400000,2849600000,1709760000,284960000,2.000000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
3,Apex Therapeutics,Molecule_085,Oncology,Ovarian Cancer,14248000000,5000400000,2849600000,1709760000,213720000,1.500000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
4,Apex Therapeutics,Molecule_105,Gastroenterology,Irritable Bowel Syndrome,14248000000,5000400000,2849600000,1709760000,284960000,2.000000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000


In [22]:
# Get all columns ending with '_Ratio'
ratio_cols = [col for col in df.columns if col.endswith('_Ratio')]

# Compute row-wise average across all ratio columns
df['Revenue_Ratio'] = df[ratio_cols].mean(axis=1)

# Find the index of the first ratio column
first_ratio_idx = df.columns.get_loc(ratio_cols[0])

# Reorder columns to insert Avg_Ratio at the desired position
cols = df.columns.tolist()

# take out Avg_Ratio from wherever it is. Put it right before the first ratio column
cols.insert(first_ratio_idx, cols.pop(cols.index('Revenue_Ratio')))

df = df[cols]

df.head()

,Company Name,Molecule Name,Therapuetic Area,Indication,2025_Company Revenue,2025_Company Cost of goods sold,2025_Company SGA,2025_Company Marketing,2025_Molecule Revenue,Revenue_Ratio,...,2021_Company SGA,2021_Company Marketing,2021_Molecule Revenue,2021_Ratio,2020_Company Revenue,2020_Company Cost of goods sold,2020_Company SGA,2020_Company Marketing,2020_Molecule Revenue,2020_Ratio
0,Apex Therapeutics,Molecule_025,Rare Diseases,Lysosomal Storage Disorders,14248000000,5000400000,2849600000,1709760000,142480000,1.416667,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
1,Apex Therapeutics,Molecule_045,Neurology,Epilepsy,14248000000,5000400000,2849600000,1709760000,213720000,1.500000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
2,Apex Therapeutics,Molecule_065,Dermatology,Atopic Dermatitis,14248000000,5000400000,2849600000,1709760000,284960000,1.583333,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
3,Apex Therapeutics,Molecule_085,Oncology,Ovarian Cancer,14248000000,5000400000,2849600000,1709760000,213720000,1.500000,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000
4,Apex Therapeutics,Molecule_105,Gastroenterology,Irritable Bowel Syndrome,14248000000,5000400000,2849600000,1709760000,284960000,1.583333,...,2673090000,1527480000,190935000,1.500000,8460000000,3891600000,1522800000,1015200000,126900000,1.500000


In [23]:
# list unique therapeutic areas
unique_areas = df['Therapuetic Area'].unique().tolist()
unique_areas

['Rare Diseases',
 'Neurology',
 'Dermatology',
 'Oncology',
 'Gastroenterology',
 'Nephrology',
 'Infectious Diseases',
 'Cardiology',
 'Ophthalmology',
 'Rheumatology',
 'Endocrinology',
 'Psychiatry',
 'Immunology',
 'Pulmonology']

In [24]:
# sort the data to group by therapuetic area and molecule name
df = df.sort_values(by=['Therapuetic Area', 'Molecule Name']).reset_index(drop=True)
df.head(20)

,Company Name,Molecule Name,Therapuetic Area,Indication,2025_Company Revenue,2025_Company Cost of goods sold,2025_Company SGA,2025_Company Marketing,2025_Molecule Revenue,Revenue_Ratio,...,2021_Company SGA,2021_Company Marketing,2021_Molecule Revenue,2021_Ratio,2020_Company Revenue,2020_Company Cost of goods sold,2020_Company SGA,2020_Company Marketing,2020_Molecule Revenue,2020_Ratio
0,BioSynth Corp.,Molecule_002,Cardiology,Hypertension,6675000000,3204000000,1468500000,500625000,38047500,1.295000,...,1486540000,675700000,67570000,1.000000,9343000000,3830630000,2148890000,1027730000,112116000,1.200000
1,Fusion Biotics,Molecule_016,Cardiology,Coronary Artery Disease,6675000000,3204000000,1468500000,500625000,66750000,1.366667,...,1486540000,675700000,67570000,1.000000,9343000000,3830630000,2148890000,1027730000,112116000,1.200000
2,Integra LifeSciences,Molecule_030,Cardiology,Heart Failure,9579000000,4406240000,2298960000,1149480000,95790000,1.166667,...,1272040000,578200000,57820000,1.000000,13546000000,5012020000,3115580000,1354600000,162552000,1.200000
3,Global BioPharma,Molecule_044,Cardiology,Atrial Fibrillation,5081000000,2184830000,1067010000,609720000,35567000,1.400000,...,1823580000,1215720000,121572000,1.200000,6753000000,2768730000,1350600000,810360000,101295000,1.500000
4,Veridian Pharma,Molecule_058,Cardiology,Hypertension,5081000000,2184830000,1067010000,609720000,25405000,1.366667,...,1823580000,1215720000,121572000,1.200000,6753000000,2768730000,1350600000,810360000,101295000,1.500000
5,Frontier Pharma,Molecule_072,Cardiology,Coronary Artery Disease,5081000000,2032400000,1067010000,609720000,35567000,1.200000,...,1692000000,846000000,126900000,1.500000,7508000000,2778000000,1651760000,900960000,75080000,1.000000
6,Nova Pharma,Molecule_086,Cardiology,Atrial Fibrillation,9579000000,4406240000,2298960000,1149480000,191580000,1.333333,...,1272040000,578200000,57820000,1.000000,13546000000,5012020000,3115580000,1354600000,162552000,1.200000
7,Genesis BioPharma,Molecule_100,Cardiology,Coronary Artery Disease,9579000000,4406240000,2298960000,1149480000,95790000,1.166667,...,1272040000,578200000,57820000,1.000000,13546000000,5012020000,3115580000,1354600000,162552000,1.200000
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,Hypertension,9579000000,4406240000,2298960000,1149480000,191580000,1.333333,...,1272040000,578200000,57820000,1.000000,13546000000,5012020000,3115580000,1354600000,162552000,1.200000
9,Prime Health Solutions,Molecule_128,Cardiology,Atrial Fibrillation,5081000000,2032400000,1067010000,609720000,50810000,1.250000,...,1692000000,846000000,126900000,1.500000,7508000000,2778000000,1651760000,900960000,75080000,1.000000


In [25]:
area_avg = df.groupby('Therapuetic Area')['Revenue_Ratio'].mean().reset_index(name='area_avg')
area_avg.head(20)

,Therapuetic Area,area_avg
0,Cardiology,1.287833
1,Dermatology,1.403704
2,Endocrinology,1.305000
3,Gastroenterology,1.348333
4,Immunology,1.303333
5,Infectious Diseases,1.319194
6,Nephrology,1.365490
7,Neurology,1.344770
8,Oncology,1.378941
9,Ophthalmology,1.290000


In [26]:
#Step 2

# Converts financial values like "$1,234,567" → 1234567.0 and removes $, commas and extra spaces
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', '').strip())
    return x

# Find all columns that end with _Ratio
ratio_cols = [col for col in df.columns if col.endswith('_Ratio')]
years = [col.replace('_Ratio', '') for col in ratio_cols]

# creates column names for the current year
for year in years:
    ratio_col = f"{year}_Ratio"
    company_cogs_col = f"{year}_Company Cost of goods sold"
    molecule_cogs_col = f"{year}_Molecule_COGS"

    # Check if Company COGS column exists
    if company_cogs_col not in df.columns:
        continue

    # Clean Company COGS column if needed
    # If the column contains strings (like "$3,000,000"), clean them using clean_currency function
    # When pandas doesn’t know what type of data a column has or when it’s mixed (numbers + text)—it assigns dtype=object. So this will basically check if the column has any strings.
    if df[company_cogs_col].dtype == object:
        df[company_cogs_col] = df[company_cogs_col].apply(clean_currency)

    # Calculate Molecule COGS
    df[molecule_cogs_col] = df[company_cogs_col] / df[ratio_col]

    # Insert Molecule_COGS column right after the Ratio column
    ratio_idx = df.columns.get_loc(ratio_col)
    df.insert(ratio_idx + 1, molecule_cogs_col, df.pop(molecule_cogs_col))

# Display Ratio and Molecule_COGS columns to verify
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']
display_cols = id_cols + [col for col in df.columns if '_Ratio' in col or '_Molecule_COGS' in col]
df[display_cols].head()


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Ratio,2025_Molecule_COGS,2024_Ratio,2024_Molecule_COGS,2023_Ratio,2023_Molecule_COGS,2022_Ratio,2022_Molecule_COGS,2021_Ratio,2021_Molecule_COGS,2020_Ratio,2020_Molecule_COGS
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,0.570000,5621052631.578947,1.800000,2208511111.111111,1.800000,2047511111.111111,1.400000,2037885714.285714,1.000000,3040650000.000000,1.200000,3192191666.666667
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,1.000000,3204000000.000000,1.800000,2208511111.111111,1.800000,2047511111.111111,1.400000,2037885714.285714,1.000000,3040650000.000000,1.200000,3192191666.666667
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,1.000000,4406240000.000000,1.800000,1338000000.000000,1.000000,5845370000.000000,1.000000,4238100000.000000,1.000000,2428440000.000000,1.200000,4176683333.333333
3,Global BioPharma,Molecule_044,Cardiology,1.400000,0.700000,3121185714.285714,2.000000,1861650000.000000,1.500000,3010840000.000000,1.500000,1464960000.000000,1.200000,3630716666.666667,1.500000,1845820000.000000
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,0.500000,4369660000.000000,2.000000,1861650000.000000,1.500000,3010840000.000000,1.500000,1464960000.000000,1.200000,3630716666.666667,1.500000,1845820000.000000


In [27]:
# Step 3

# creates column names for the current year
for year in years:
    molecule_cogs_col = f"{year}_Molecule_COGS"
    molecule_revenue_col = f"{year}_Molecule Revenue"
    molecule_cogs_pct_col = f"{year}_Molecule_COGS_%_Sales"

    # Check if required columns exist
    if molecule_cogs_col not in df.columns or molecule_revenue_col not in df.columns:
        continue

    # Clean Molecule Revenue column if needed
    if df[molecule_revenue_col].dtype == object:
        df[molecule_revenue_col] = df[molecule_revenue_col].apply(clean_currency)

    # Calculate Molecule COGS % of Sales
    df[molecule_cogs_pct_col] = df[molecule_cogs_col] / df[molecule_revenue_col]

    # Insert the new column right after Molecule COGS
    cogs_idx = df.columns.get_loc(molecule_cogs_col)
    df.insert(cogs_idx + 1, molecule_cogs_pct_col, df.pop(molecule_cogs_pct_col))

# Display Molecule COGS, Molecule Revenue, and Molecule COGS % of Sales columns for quick check
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']
display_cols = id_cols + [col for col in df.columns if '_Molecule_COGS' in col or '_Molecule Revenue' in col]
df[display_cols].head()


,Company Name,Molecule Name,Therapuetic Area,2025_Molecule Revenue,2025_Molecule_COGS,2025_Molecule_COGS_%_Sales,2024_Molecule Revenue,2024_Molecule_COGS,2024_Molecule_COGS_%_Sales,2023_Molecule Revenue,...,2023_Molecule_COGS_%_Sales,2022_Molecule Revenue,2022_Molecule_COGS,2022_Molecule_COGS_%_Sales,2021_Molecule Revenue,2021_Molecule_COGS,2021_Molecule_COGS_%_Sales,2020_Molecule Revenue,2020_Molecule_COGS,2020_Molecule_COGS_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,38047500,5621052631.578947,147.737765,155556000,2208511111.111111,14.197531,144216000,...,14.197531,105112000,2037885714.285714,19.387755,67570000,3040650000.000000,45.000000,112116000,3192191666.666667,28.472222
1,Fusion Biotics,Molecule_016,Cardiology,66750000,3204000000.000000,48.000000,155556000,2208511111.111111,14.197531,144216000,...,14.197531,105112000,2037885714.285714,19.387755,67570000,3040650000.000000,45.000000,112116000,3192191666.666667,28.472222
2,Integra LifeSciences,Molecule_030,Cardiology,95790000,4406240000.000000,45.998956,108378000,1338000000.000000,12.345679,142570000,...,41.000000,94180000,4238100000.000000,45.000000,57820000,2428440000.000000,42.000000,162552000,4176683333.333333,25.694444
3,Global BioPharma,Molecule_044,Cardiology,35567000,3121185714.285714,87.755102,212760000,1861650000.000000,8.750000,161295000,...,18.666667,78480000,1464960000.000000,18.666667,121572000,3630716666.666667,29.864744,101295000,1845820000.000000,18.222222
4,Veridian Pharma,Molecule_058,Cardiology,25405000,4369660000.000000,172.000000,212760000,1861650000.000000,8.750000,161295000,...,18.666667,78480000,1464960000.000000,18.666667,121572000,3630716666.666667,29.864744,101295000,1845820000.000000,18.222222


In [28]:
# Step 4

# Drop any leftover Avg_Molecule_COGS columns
# errors='ignore': prevents crashing if no such column exists.
df = df.drop(columns=[col for col in df.columns if 'Avg_Molecule_COGS' in col], errors='ignore')

# Find all Molecule COGS columns
# Excludes columns like 2025_Molecule_COGS_%_Sales (from Step 3) to avoid incorrect averages.
molecule_cogs_cols = [col for col in df.columns if '_Molecule_COGS' in col and not '_%_Sales' in col]

# Calculate average Molecule COGS for each molecule
df['Avg_Molecule_COGS'] = df[molecule_cogs_cols].mean(axis=1)

# Insert Avg_Molecule_COGS after last Molecule COGS
last_cogs_idx = max(df.columns.get_loc(col) for col in molecule_cogs_cols)
# takes two or more values and returns the smallest one
insert_pos = min(last_cogs_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_COGS', df.pop('Avg_Molecule_COGS'))

# Columns to display: Company, Molecule, TA + COGS values
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']
display_cols = id_cols + molecule_cogs_cols + ['Avg_Molecule_COGS']

# Verify averages with identifiers
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,2025_Molecule_COGS,2024_Molecule_COGS,2023_Molecule_COGS,2022_Molecule_COGS,2021_Molecule_COGS,2020_Molecule_COGS,Avg_Molecule_COGS
0,BioSynth Corp.,Molecule_002,Cardiology,5621052631.578947,2208511111.111111,2047511111.111111,2037885714.285714,3040650000.000000,3192191666.666667,3024633705.792259
1,Fusion Biotics,Molecule_016,Cardiology,3204000000.000000,2208511111.111111,2047511111.111111,2037885714.285714,3040650000.000000,3192191666.666667,2621791600.529100
2,Integra LifeSciences,Molecule_030,Cardiology,4406240000.000000,1338000000.000000,5845370000.000000,4238100000.000000,2428440000.000000,4176683333.333333,3738805555.555555
3,Global BioPharma,Molecule_044,Cardiology,3121185714.285714,1861650000.000000,3010840000.000000,1464960000.000000,3630716666.666667,1845820000.000000,2489195396.825397
4,Veridian Pharma,Molecule_058,Cardiology,4369660000.000000,1861650000.000000,3010840000.000000,1464960000.000000,3630716666.666667,1845820000.000000,2697274444.444445
5,Frontier Pharma,Molecule_072,Cardiology,2903428571.428571,2448680000.000000,2312800000.000000,3341346666.666667,2481600000.000000,2778000000.000000,2710975873.015873
6,Nova Pharma,Molecule_086,Cardiology,2203120000.000000,1338000000.000000,5845370000.000000,4238100000.000000,2428440000.000000,4176683333.333333,3371618888.888889
7,Genesis BioPharma,Molecule_100,Cardiology,4406240000.000000,1338000000.000000,5845370000.000000,4238100000.000000,2428440000.000000,4176683333.333333,3738805555.555555
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,2203120000.000000,1338000000.000000,5845370000.000000,4238100000.000000,2428440000.000000,4176683333.333333,3371618888.888889
9,Prime Health Solutions,Molecule_128,Cardiology,2032400000.000000,2448680000.000000,2312800000.000000,3341346666.666667,2481600000.000000,2778000000.000000,2565804444.444444


In [29]:
#Step 5

# Drop any leftover Avg_Molecule_COGS_%_Sales columns
df = df.drop(columns=[col for col in df.columns if 'Avg_Molecule_COGS_%_Sales' in col], errors='ignore')

# Find all Molecule COGS % of Sales columns
molecule_cogs_pct_cols = [col for col in df.columns if '_Molecule_COGS_%_Sales' in col]

# Calculate average Molecule COGS % of Sales for each molecule
df['Avg_Molecule_COGS_%_Sales'] = df[molecule_cogs_pct_cols].mean(axis=1)

# Insert Avg_Molecule_COGS_%_Sales neatly after last Molecule_COGS_%_Sales column
last_pct_idx = max(df.columns.get_loc(col) for col in molecule_cogs_pct_cols)
insert_pos = min(last_pct_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_COGS_%_Sales', df.pop('Avg_Molecule_COGS_%_Sales'))

# Columns to display: Company, Molecule, TA + Molecule COGS % of Sales values
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']
display_cols = id_cols + molecule_cogs_pct_cols + ['Avg_Molecule_COGS_%_Sales']

# Verify averages with identifiers
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,2025_Molecule_COGS_%_Sales,2024_Molecule_COGS_%_Sales,2023_Molecule_COGS_%_Sales,2022_Molecule_COGS_%_Sales,2021_Molecule_COGS_%_Sales,2020_Molecule_COGS_%_Sales,Avg_Molecule_COGS_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,147.737765,14.197531,14.197531,19.387755,45.000000,28.472222,44.832134
1,Fusion Biotics,Molecule_016,Cardiology,48.000000,14.197531,14.197531,19.387755,45.000000,28.472222,28.209173
2,Integra LifeSciences,Molecule_030,Cardiology,45.998956,12.345679,41.000000,45.000000,42.000000,25.694444,35.339847
3,Global BioPharma,Molecule_044,Cardiology,87.755102,8.750000,18.666667,18.666667,29.864744,18.222222,30.320900
4,Veridian Pharma,Molecule_058,Cardiology,172.000000,8.750000,18.666667,18.666667,29.864744,18.222222,44.361717
5,Frontier Pharma,Molecule_072,Cardiology,81.632653,17.333333,40.000000,16.444444,19.555556,37.000533,35.327753
6,Nova Pharma,Molecule_086,Cardiology,11.499739,12.345679,41.000000,45.000000,42.000000,25.694444,29.589977
7,Genesis BioPharma,Molecule_100,Cardiology,45.998956,12.345679,41.000000,45.000000,42.000000,25.694444,35.339847
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,11.499739,12.345679,41.000000,45.000000,42.000000,25.694444,29.589977
9,Prime Health Solutions,Molecule_128,Cardiology,40.000000,17.333333,40.000000,16.444444,19.555556,37.000533,28.388978


In [30]:
#SGA

# Molecule SGA

# Clean currency values function (reuse from earlier)
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', '').strip())
    return x

# Find all years dynamically from Ratio columns
ratio_cols = [col for col in df.columns if col.endswith('_Ratio')]
years = [col.replace('_Ratio', '') for col in ratio_cols]

for year in years:
    ratio_col = f"{year}_Ratio"
    company_sga_col = f"{year}_Company SGA"
    molecule_sga_col = f"{year}_Molecule_SGA"

    # Check if Company SGA column exists
    if company_sga_col not in df.columns:
        continue

    # Clean Company SGA column if needed
    if df[company_sga_col].dtype == object:
        df[company_sga_col] = df[company_sga_col].apply(clean_currency)

    # Calculate Molecule SGA
    df[molecule_sga_col] = df[company_sga_col] / df[ratio_col]

    # Insert Molecule_SGA column right after the Ratio column
    ratio_idx = df.columns.get_loc(ratio_col)
    df.insert(ratio_idx + 1, molecule_sga_col, df.pop(molecule_sga_col))

# Display Ratio and Molecule SGA columns
yearly_cols = []
for year in years:
    company_sga_col = f"{year}_Company SGA"
    ratio_col = f"{year}_Ratio"
    molecule_sga_col = f"{year}_Molecule_SGA"

    # Only add if columns exist
    cols_for_year = [col for col in [company_sga_col, ratio_col, molecule_sga_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

# Combine identifiers + yearly grouped columns
display_cols = id_cols + yearly_cols
df[display_cols].head()


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company SGA,2025_Ratio,2025_Molecule_SGA,2024_Company SGA,2024_Ratio,2024_Molecule_SGA,...,2023_Molecule_SGA,2022_Company SGA,2022_Ratio,2022_Molecule_SGA,2021_Company SGA,2021_Ratio,2021_Molecule_SGA,2020_Company SGA,2020_Ratio,2020_Molecule_SGA
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,1468500000,0.570000,2576315789.473684,1555560000,1.800000,864200000.000000,...,890222222.222222,1651760000,1.400000,1179828571.428571,1486540000,1.000000,1486540000.000000,2148890000,1.200000,1790741666.666667
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,1468500000,1.000000,1468500000.000000,1555560000,1.800000,864200000.000000,...,890222222.222222,1651760000,1.400000,1179828571.428571,1486540000,1.000000,1486540000.000000,2148890000,1.200000,1790741666.666667
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,1264410000,1.800000,702450000.000000,...,2566260000.000000,1695240000,1.000000,1695240000.000000,1272040000,1.000000,1272040000.000000,3115580000,1.200000,2596316666.666667
3,Global BioPharma,Molecule_044,Cardiology,1.400000,1067010000,0.700000,1524300000.000000,2233980000,2.000000,1116990000.000000,...,1648800000.000000,1255680000,1.500000,837120000.000000,1823580000,1.200000,1519650000.000000,1350600000,1.500000,900400000.000000
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,1067010000,0.500000,2134020000.000000,2233980000,2.000000,1116990000.000000,...,1648800000.000000,1255680000,1.500000,837120000.000000,1823580000,1.200000,1519650000.000000,1350600000,1.500000,900400000.000000


In [31]:
# Molecule SGA % of Sales
for year in years:
    molecule_sga_col = f"{year}_Molecule_SGA"
    molecule_revenue_col = f"{year}_Molecule Revenue"
    molecule_sga_pct_col = f"{year}_Molecule_SGA_%_Sales"

    # Check if required columns exist
    if molecule_sga_col not in df.columns or molecule_revenue_col not in df.columns:
        continue

    # Clean Molecule Revenue column if needed
    if df[molecule_revenue_col].dtype == object:
        df[molecule_revenue_col] = df[molecule_revenue_col].apply(clean_currency)

    # Calculate Molecule SGA % of Sales
    df[molecule_sga_pct_col] = df[molecule_sga_col] / df[molecule_revenue_col]

    # Insert the new column right after Molecule SGA
    sga_idx = df.columns.get_loc(molecule_sga_col)
    df.insert(sga_idx + 1, molecule_sga_pct_col, df.pop(molecule_sga_pct_col))

# Display identifiers + all SGA-related columns grouped by year
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']

# Build a list of columns grouped by year
yearly_cols = []
for year in years:
    company_sga_col = f"{year}_Company SGA"
    ratio_col = f"{year}_Ratio"
    molecule_sga_col = f"{year}_Molecule_SGA"
    molecule_sga_pct_col = f"{year}_Molecule_SGA_%_Sales"

    # Only add if columns exist
    cols_for_year = [col for col in [company_sga_col, ratio_col, molecule_sga_col, molecule_sga_pct_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

# Combine identifiers + yearly grouped columns
display_cols = id_cols + yearly_cols

# Preview the data
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company SGA,2025_Ratio,2025_Molecule_SGA,2025_Molecule_SGA_%_Sales,2024_Company SGA,2024_Ratio,...,2022_Molecule_SGA,2022_Molecule_SGA_%_Sales,2021_Company SGA,2021_Ratio,2021_Molecule_SGA,2021_Molecule_SGA_%_Sales,2020_Company SGA,2020_Ratio,2020_Molecule_SGA,2020_Molecule_SGA_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,1468500000,0.570000,2576315789.473684,67.713143,1555560000,1.800000,...,1179828571.428571,11.224490,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,1468500000,1.000000,1468500000.000000,22.000000,1555560000,1.800000,...,1179828571.428571,11.224490,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,1695240000.000000,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222
3,Global BioPharma,Molecule_044,Cardiology,1.400000,1067010000,0.700000,1524300000.000000,42.857143,2233980000,2.000000,...,837120000.000000,10.666667,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,1067010000,0.500000,2134020000.000000,84.000000,2233980000,2.000000,...,837120000.000000,10.666667,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,1067010000,0.700000,1524300000.000000,42.857143,1601060000,1.500000,...,1806133333.333333,8.888889,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000
6,Nova Pharma,Molecule_086,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,1695240000.000000,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,1695240000.000000,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,1695240000.000000,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,1067010000,1.000000,1067010000.000000,21.000000,1601060000,1.500000,...,1806133333.333333,8.888889,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000


In [32]:
# Avg Molecule SGA

# Drop any leftover Avg_Molecule_SGA columns from previous runs
df = df.drop(columns=[col for col in df.columns if 'Avg_Molecule_SGA' in col], errors='ignore')

# Find all Molecule SGA columns dynamically (excluding %_Sales)
molecule_sga_cols = [col for col in df.columns if '_Molecule_SGA' in col and '_%_Sales' not in col]

# Calculate average Molecule SGA for each molecule
df['Avg_Molecule_SGA'] = df[molecule_sga_cols].mean(axis=1)

# Insert Avg_Molecule_SGA neatly after the last Molecule SGA column
last_sga_idx = max(df.columns.get_loc(col) for col in molecule_sga_cols)
insert_pos = min(last_sga_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_SGA', df.pop('Avg_Molecule_SGA'))

# Display identifiers + all SGA-related columns grouped by year + Avg Molecule SGA
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']

# Build a list of columns grouped by year
yearly_cols = []
for year in years:
    company_sga_col = f"{year}_Company SGA"
    ratio_col = f"{year}_Ratio"
    molecule_sga_col = f"{year}_Molecule_SGA"
    molecule_sga_pct_col = f"{year}_Molecule_SGA_%_Sales"

    # Only add if columns exist
    cols_for_year = [col for col in [company_sga_col, ratio_col, molecule_sga_col, molecule_sga_pct_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

# Combine identifiers + yearly grouped columns + Avg Molecule SGA
display_cols = id_cols + yearly_cols + ['Avg_Molecule_SGA']

# Preview the data
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company SGA,2025_Ratio,2025_Molecule_SGA,2025_Molecule_SGA_%_Sales,2024_Company SGA,2024_Ratio,...,2022_Molecule_SGA_%_Sales,2021_Company SGA,2021_Ratio,2021_Molecule_SGA,2021_Molecule_SGA_%_Sales,2020_Company SGA,2020_Ratio,2020_Molecule_SGA,2020_Molecule_SGA_%_Sales,Avg_Molecule_SGA
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,1468500000,0.570000,2576315789.473684,67.713143,1555560000,1.800000,...,11.224490,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222,1464641374.965191
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,1468500000,1.000000,1468500000.000000,22.000000,1555560000,1.800000,...,11.224490,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222,1280005410.052910
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1855211111.111111
3,Global BioPharma,Molecule_044,Cardiology,1.400000,1067010000,0.700000,1524300000.000000,42.857143,2233980000,2.000000,...,10.666667,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889,1257876666.666667
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,1067010000,0.500000,2134020000.000000,84.000000,2233980000,2.000000,...,10.666667,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889,1359496666.666667
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,1067010000,0.700000,1524300000.000000,42.857143,1601060000,1.500000,...,8.888889,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000,1388994444.444444
6,Nova Pharma,Molecule_086,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1663631111.111111
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1855211111.111111
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,18.000000,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1663631111.111111
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,1067010000,1.000000,1067010000.000000,21.000000,1601060000,1.500000,...,8.888889,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000,1312779444.444444


In [33]:
# Avg Molecule SGA % of sales

# Drop any leftover Avg_Molecule_SGA_%_Sales columns from previous runs
df = df.drop(columns=[col for col in df.columns if 'Avg_Molecule_SGA_%_Sales' in col], errors='ignore')

# Find all Molecule SGA % of Sales columns dynamically
molecule_sga_pct_cols = [col for col in df.columns if '_Molecule_SGA_%_Sales' in col]

# Calculate average Molecule SGA % of Sales for each molecule
df['Avg_Molecule_SGA_%_Sales'] = df[molecule_sga_pct_cols].mean(axis=1)

# Insert Avg_Molecule_SGA_%_Sales neatly after last Molecule_SGA_%_Sales column
last_pct_idx = max(df.columns.get_loc(col) for col in molecule_sga_pct_cols)
insert_pos = min(last_pct_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_SGA_%_Sales', df.pop('Avg_Molecule_SGA_%_Sales'))

# Display identifiers + all SGA-related columns grouped by year + both averages
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']

# Build a list of columns grouped by year
yearly_cols = []
for year in years:
    company_sga_col = f"{year}_Company SGA"
    ratio_col = f"{year}_Ratio"
    molecule_sga_col = f"{year}_Molecule_SGA"
    molecule_sga_pct_col = f"{year}_Molecule_SGA_%_Sales"

    # Only add if columns exist
    cols_for_year = [col for col in [company_sga_col, ratio_col, molecule_sga_col, molecule_sga_pct_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

# Combine identifiers + yearly grouped columns + both averages
display_cols = id_cols + yearly_cols + ['Avg_Molecule_SGA', 'Avg_Molecule_SGA_%_Sales']

# Preview the data
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company SGA,2025_Ratio,2025_Molecule_SGA,2025_Molecule_SGA_%_Sales,2024_Company SGA,2024_Ratio,...,2021_Company SGA,2021_Ratio,2021_Molecule_SGA,2021_Molecule_SGA_%_Sales,2020_Company SGA,2020_Ratio,2020_Molecule_SGA,2020_Molecule_SGA_%_Sales,Avg_Molecule_SGA,Avg_Molecule_SGA_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,1468500000,0.570000,2576315789.473684,67.713143,1555560000,1.800000,...,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222,1464641374.965191,21.439708
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,1468500000,1.000000,1468500000.000000,22.000000,1555560000,1.800000,...,1486540000,1.000000,1486540000.000000,22.000000,2148890000,1.200000,1790741666.666667,15.972222,1280005410.052910,13.820851
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1855211111.111111,17.408951
3,Global BioPharma,Molecule_044,Cardiology,1.400000,1067010000,0.700000,1524300000.000000,42.857143,2233980000,2.000000,...,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889,1257876666.666667,15.064160
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,1067010000,0.500000,2134020000.000000,84.000000,2233980000,2.000000,...,1823580000,1.200000,1519650000.000000,12.500000,1350600000,1.500000,900400000.000000,8.888889,1359496666.666667,21.921303
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,1067010000,0.700000,1524300000.000000,42.857143,1601060000,1.500000,...,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000,1388994444.444444,18.365079
6,Nova Pharma,Molecule_086,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1663631111.111111,14.408951
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,2298960000,1.000000,2298960000.000000,24.000000,1264410000,1.800000,...,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1855211111.111111,17.408951
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,2298960000,2.000000,1149480000.000000,6.000000,1264410000,1.800000,...,1272040000,1.000000,1272040000.000000,22.000000,3115580000,1.200000,2596316666.666667,15.972222,1663631111.111111,14.408951
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,1067010000,1.000000,1067010000.000000,21.000000,1601060000,1.500000,...,1692000000,1.500000,1128000000.000000,8.888889,1651760000,1.000000,1651760000.000000,22.000000,1312779444.444444,14.722222


In [34]:
# Molecule Marketing

# Drop any leftover Avg_Molecule_Marketing columns from previous runs
df = df.drop(columns=[col for col in df.columns if 'Molecule_Marketing' in col], errors='ignore')

for year in years:
    ratio_col = f"{year}_Ratio"
    company_marketing_col = f"{year}_Company Marketing"
    molecule_marketing_col = f"{year}_Molecule_Marketing"

    # Check if Company Marketing column exists
    if company_marketing_col not in df.columns:
        continue

    # Clean Company Marketing column if needed
    if df[company_marketing_col].dtype == object:
        df[company_marketing_col] = df[company_marketing_col].apply(clean_currency)

    # Calculate Molecule Marketing
    df[molecule_marketing_col] = df[company_marketing_col] / df[ratio_col]

    # Insert Molecule_Marketing column right after the Ratio column
    ratio_idx = df.columns.get_loc(ratio_col)
    df.insert(ratio_idx + 1, molecule_marketing_col, df.pop(molecule_marketing_col))

# Display identifiers + Company Marketing + Ratios + Molecule Marketing grouped per year
id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area']

# Build a list of columns grouped by year
yearly_cols = []
for year in years:
    company_marketing_col = f"{year}_Company Marketing"
    ratio_col = f"{year}_Ratio"
    molecule_marketing_col = f"{year}_Molecule_Marketing"

    cols_for_year = [col for col in [company_marketing_col, ratio_col, molecule_marketing_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

display_cols = id_cols + yearly_cols
df[display_cols].head(10)

,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company Marketing,2025_Ratio,2025_Molecule_Marketing,2024_Company Marketing,2024_Ratio,2024_Molecule_Marketing,...,2023_Molecule_Marketing,2022_Company Marketing,2022_Ratio,2022_Molecule_Marketing,2021_Company Marketing,2021_Ratio,2021_Molecule_Marketing,2020_Company Marketing,2020_Ratio,2020_Molecule_Marketing
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,500625000,0.570000,878289473.684210,1037040000,1.800000,576133333.333333,...,534133333.333333,900960000,1.400000,643542857.142857,675700000,1.000000,675700000.000000,1027730000,1.200000,856441666.666667
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,500625000,1.000000,500625000.000000,1037040000,1.800000,576133333.333333,...,534133333.333333,900960000,1.400000,643542857.142857,675700000,1.000000,675700000.000000,1027730000,1.200000,856441666.666667
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,662310000,1.800000,367950000.000000,...,1425700000.000000,1130160000,1.000000,1130160000.000000,578200000,1.000000,578200000.000000,1354600000,1.200000,1128833333.333333
3,Global BioPharma,Molecule_044,Cardiology,1.400000,609720000,0.700000,871028571.428571,1489320000,2.000000,744660000.000000,...,860240000.000000,627840000,1.500000,418560000.000000,1215720000,1.200000,1013100000.000000,810360000,1.500000,540240000.000000
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,609720000,0.500000,1219440000.000000,1489320000,2.000000,744660000.000000,...,860240000.000000,627840000,1.500000,418560000.000000,1215720000,1.200000,1013100000.000000,810360000,1.500000,540240000.000000
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,609720000,0.700000,871028571.428571,1130160000,1.500000,753440000.000000,...,693840000.000000,1354600000,1.500000,903066666.666667,846000000,1.500000,564000000.000000,900960000,1.000000,900960000.000000
6,Nova Pharma,Molecule_086,Cardiology,1.333333,1149480000,2.000000,574740000.000000,662310000,1.800000,367950000.000000,...,1425700000.000000,1130160000,1.000000,1130160000.000000,578200000,1.000000,578200000.000000,1354600000,1.200000,1128833333.333333
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,662310000,1.800000,367950000.000000,...,1425700000.000000,1130160000,1.000000,1130160000.000000,578200000,1.000000,578200000.000000,1354600000,1.200000,1128833333.333333
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,1149480000,2.000000,574740000.000000,662310000,1.800000,367950000.000000,...,1425700000.000000,1130160000,1.000000,1130160000.000000,578200000,1.000000,578200000.000000,1354600000,1.200000,1128833333.333333
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,609720000,1.000000,609720000.000000,1130160000,1.500000,753440000.000000,...,693840000.000000,1354600000,1.500000,903066666.666667,846000000,1.500000,564000000.000000,900960000,1.000000,900960000.000000


In [35]:
# Molecule Marketing % of Sales
for year in years:
    molecule_marketing_col = f"{year}_Molecule_Marketing"
    molecule_revenue_col = f"{year}_Molecule Revenue"
    molecule_marketing_pct_col = f"{year}_Molecule_Marketing_%_Sales"

    if molecule_marketing_col not in df.columns or molecule_revenue_col not in df.columns:
        continue

    if df[molecule_revenue_col].dtype == object:
        df[molecule_revenue_col] = df[molecule_revenue_col].apply(clean_currency)

    df[molecule_marketing_pct_col] = df[molecule_marketing_col] / df[molecule_revenue_col]

    marketing_idx = df.columns.get_loc(molecule_marketing_col)
    df.insert(marketing_idx + 1, molecule_marketing_pct_col, df.pop(molecule_marketing_pct_col))

# Update display with % of Sales columns grouped per year
yearly_cols = []
for year in years:
    company_marketing_col = f"{year}_Company Marketing"
    ratio_col = f"{year}_Ratio"
    molecule_marketing_col = f"{year}_Molecule_Marketing"
    molecule_marketing_pct_col = f"{year}_Molecule_Marketing_%_Sales"

    cols_for_year = [col for col in [company_marketing_col, ratio_col, molecule_marketing_col, molecule_marketing_pct_col] if col in df.columns]
    yearly_cols.extend(cols_for_year)

display_cols = id_cols + yearly_cols
df[display_cols].head(10)

,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company Marketing,2025_Ratio,2025_Molecule_Marketing,2025_Molecule_Marketing_%_Sales,2024_Company Marketing,2024_Ratio,...,2022_Molecule_Marketing,2022_Molecule_Marketing_%_Sales,2021_Company Marketing,2021_Ratio,2021_Molecule_Marketing,2021_Molecule_Marketing_%_Sales,2020_Company Marketing,2020_Ratio,2020_Molecule_Marketing,2020_Molecule_Marketing_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,500625000,0.570000,878289473.684210,23.084026,1037040000,1.800000,...,643542857.142857,6.122449,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,500625000,1.000000,500625000.000000,7.500000,1037040000,1.800000,...,643542857.142857,6.122449,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,1130160000.000000,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444
3,Global BioPharma,Molecule_044,Cardiology,1.400000,609720000,0.700000,871028571.428571,24.489796,1489320000,2.000000,...,418560000.000000,5.333333,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,609720000,0.500000,1219440000.000000,48.000000,1489320000,2.000000,...,418560000.000000,5.333333,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,609720000,0.700000,871028571.428571,24.489796,1130160000,1.500000,...,903066666.666667,4.444444,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000
6,Nova Pharma,Molecule_086,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,1130160000.000000,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,1130160000.000000,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,1130160000.000000,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,609720000,1.000000,609720000.000000,12.000000,1130160000,1.500000,...,903066666.666667,4.444444,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000


In [36]:
# Avg Molecule Marketing

# Find all Molecule Marketing columns (exclude %_Sales)
molecule_marketing_cols = [col for col in df.columns if '_Molecule_Marketing' in col and '_%_Sales' not in col]

df['Avg_Molecule_Marketing'] = df[molecule_marketing_cols].mean(axis=1)

# Insert Avg_Molecule_Marketing neatly after last Molecule Marketing column
last_marketing_idx = max(df.columns.get_loc(col) for col in molecule_marketing_cols)
insert_pos = min(last_marketing_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_Marketing', df.pop('Avg_Molecule_Marketing'))

# Add Avg to display
display_cols += ['Avg_Molecule_Marketing']
df[display_cols].head(10)


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company Marketing,2025_Ratio,2025_Molecule_Marketing,2025_Molecule_Marketing_%_Sales,2024_Company Marketing,2024_Ratio,...,2022_Molecule_Marketing_%_Sales,2021_Company Marketing,2021_Ratio,2021_Molecule_Marketing,2021_Molecule_Marketing_%_Sales,2020_Company Marketing,2020_Ratio,2020_Molecule_Marketing,2020_Molecule_Marketing_%_Sales,Avg_Molecule_Marketing
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,500625000,0.570000,878289473.684210,23.084026,1037040000,1.800000,...,6.122449,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889,694040110.693400
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,500625000,1.000000,500625000.000000,7.500000,1037040000,1.800000,...,6.122449,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889,631096031.746032
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,963387222.222222
3,Global BioPharma,Molecule_044,Cardiology,1.400000,609720000,0.700000,871028571.428571,24.489796,1489320000,2.000000,...,5.333333,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333,741304761.904762
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,609720000,0.500000,1219440000.000000,48.000000,1489320000,2.000000,...,5.333333,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333,799373333.333333
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,609720000,0.700000,871028571.428571,24.489796,1130160000,1.500000,...,4.444444,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000,781055873.015873
6,Nova Pharma,Molecule_086,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,867597222.222222
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,963387222.222222
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,12.000000,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,867597222.222222
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,609720000,1.000000,609720000.000000,12.000000,1130160000,1.500000,...,4.444444,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000,737504444.444444


In [37]:
# Avg Molecule Marketing % of Sales

# Find all Molecule Marketing % of Sales columns
molecule_marketing_pct_cols = [col for col in df.columns if '_Molecule_Marketing_%_Sales' in col]

df['Avg_Molecule_Marketing_%_Sales'] = df[molecule_marketing_pct_cols].mean(axis=1)

# Insert Avg_Molecule_Marketing_%_Sales neatly after last Molecule Marketing % of Sales column
last_pct_idx = max(df.columns.get_loc(col) for col in molecule_marketing_pct_cols)
insert_pos = min(last_pct_idx + 1, len(df.columns))
df.insert(insert_pos, 'Avg_Molecule_Marketing_%_Sales', df.pop('Avg_Molecule_Marketing_%_Sales'))

# Add Avg % of Sales to display
display_cols += ['Avg_Molecule_Marketing_%_Sales']
df[display_cols].head(20)


,Company Name,Molecule Name,Therapuetic Area,Revenue_Ratio,2025_Company Marketing,2025_Ratio,2025_Molecule_Marketing,2025_Molecule_Marketing_%_Sales,2024_Company Marketing,2024_Ratio,...,2021_Company Marketing,2021_Ratio,2021_Molecule_Marketing,2021_Molecule_Marketing_%_Sales,2020_Company Marketing,2020_Ratio,2020_Molecule_Marketing,2020_Molecule_Marketing_%_Sales,Avg_Molecule_Marketing,Avg_Molecule_Marketing_%_Sales
0,BioSynth Corp.,Molecule_002,Cardiology,1.295000,500625000,0.570000,878289473.684210,23.084026,1037040000,1.800000,...,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889,694040110.693400,9.042129
1,Fusion Biotics,Molecule_016,Cardiology,1.366667,500625000,1.000000,500625000.000000,7.500000,1037040000,1.800000,...,675700000,1.000000,675700000.000000,10.000000,1027730000,1.200000,856441666.666667,7.638889,631096031.746032,6.444791
2,Integra LifeSciences,Molecule_030,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,963387222.222222,9.056584
3,Global BioPharma,Molecule_044,Cardiology,1.400000,609720000,0.700000,871028571.428571,24.489796,1489320000,2.000000,...,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333,741304761.904762,8.720522
4,Veridian Pharma,Molecule_058,Cardiology,1.366667,609720000,0.500000,1219440000.000000,48.000000,1489320000,2.000000,...,1215720000,1.200000,1013100000.000000,8.333333,810360000,1.500000,540240000.000000,5.333333,799373333.333333,12.638889
5,Frontier Pharma,Molecule_072,Cardiology,1.200000,609720000,0.700000,871028571.428571,24.489796,1130160000,1.500000,...,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000,781055873.015873,10.452003
6,Nova Pharma,Molecule_086,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,867597222.222222,7.556584
7,Genesis BioPharma,Molecule_100,Cardiology,1.166667,1149480000,1.000000,1149480000.000000,12.000000,662310000,1.800000,...,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,963387222.222222,9.056584
8,Pinnacle Pharmaceuticals,Molecule_114,Cardiology,1.333333,1149480000,2.000000,574740000.000000,3.000000,662310000,1.800000,...,578200000,1.000000,578200000.000000,10.000000,1354600000,1.200000,1128833333.333333,6.944444,867597222.222222,7.556584
9,Prime Health Solutions,Molecule_128,Cardiology,1.250000,609720000,1.000000,609720000.000000,12.000000,1130160000,1.500000,...,846000000,1.500000,564000000.000000,4.444444,900960000,1.000000,900960000.000000,12.000000,737504444.444444,8.370370


In [38]:
# list unique therapeutic areas
unique_areas = df['Indication'].unique().tolist()
unique_areas

['Hypertension',
 'Coronary Artery Disease',
 'Heart Failure',
 'Atrial Fibrillation',
 'Eczema',
 'Psoriasis',
 'Acne',
 'Atopic Dermatitis',
 'Diabetes Mellitus Type 2',
 'Diabetes Mellitus Type 1',
 'Hypothyroidism',
 'Osteoporosis',
 'Adrenal Insufficiency',
 'Irritable Bowel Syndrome',
 'Fatty Liver Disease',
 'Celiac Disease',
 'Rheumatoid Arthritis',
 "Crohn's Disease",
 'Lupus',
 'Ulcerative Colitis',
 'Hepatitis C',
 'Influenza',
 'Bacterial Infections',
 'Viral Infections',
 'Sepsis',
 'Chronic Kidney Disease',
 'Polycystic Kidney Disease',
 'Kidney Stones',
 "Alzheimer's Disease",
 "Parkinson's Disease",
 'Multiple Sclerosis',
 'Epilepsy',
 'Migraine',
 'Breast Cancer',
 'Lung Cancer',
 'Colorectal Cancer',
 'Melanoma',
 'Prostate Cancer',
 'Ovarian Cancer',
 'Macular Degeneration',
 'Glaucoma',
 'Diabetic Retinopathy',
 'Major Depressive Disorder',
 'Schizophrenia',
 'Bipolar Disorder',
 'Anxiety Disorder',
 'PTSD',
 'Asthma',
 'COPD',
 'Cystic Fibrosis',
 'Pulmonary Fibros

In [39]:
# Ask user for search type
search_type = input("Do you want to search by Molecule Name (M) or Other (Therapeutic Area/Indication) (O)? Enter 'M' or 'O': ").strip().upper()

if search_type == 'M':
    # Search by Molecule Name
    molecule_input = input("Enter Molecule Name: ").strip()

    # Filter dataframe for the entered molecule (case-insensitive match)
    filtered_df = df[df['Molecule Name'].str.lower() == molecule_input.lower()]

    if filtered_df.empty:
        print(f"No data found for Molecule: {molecule_input}")
    else:
        # Identify columns to display
        id_cols = ['Company Name', 'Molecule Name', 'Therapuetic Area', 'Indication']
        avg_cols = [
            'Avg_Molecule_COGS', 'Avg_Molecule_SGA', 'Avg_Molecule_Marketing',
            'Avg_Molecule_COGS_%_Sales', 'Avg_Molecule_SGA_%_Sales', 'Avg_Molecule_Marketing_%_Sales'
        ]

        avg_cols_existing = [col for col in avg_cols if col in df.columns]
        display_cols = id_cols + avg_cols_existing

        # Display results
        display(filtered_df[display_cols])

elif search_type == 'O':
    # Search by Therapeutic Area or Indication
    ta_or_ind_input = input("Enter Therapeutic Area or Indication: ").strip()

    # Filter dataframe for rows matching Therapeutic Area or Indication (case-insensitive, partial match)
    filtered_df = df[
        df['Therapuetic Area'].str.lower().str.contains(ta_or_ind_input.lower(), na=False) |
        df['Indication'].str.lower().str.contains(ta_or_ind_input.lower(), na=False)
    ]

    if filtered_df.empty:
        print(f"No data found for Therapeutic Area or Indication: {ta_or_ind_input}")
    else:
        # Identify columns to calculate averages
        avg_cols = [
            'Avg_Molecule_COGS', 'Avg_Molecule_SGA', 'Avg_Molecule_Marketing',
            'Avg_Molecule_COGS_%_Sales', 'Avg_Molecule_SGA_%_Sales', 'Avg_Molecule_Marketing_%_Sales'
        ]
        avg_cols_existing = [col for col in avg_cols if col in df.columns]

        # Calculate averages across all filtered molecules
        ta_averages = filtered_df[avg_cols_existing].mean().to_frame().T
        ta_averages.insert(0, 'Search Term', ta_or_ind_input.title())
        ta_averages.insert(1, 'Number of Molecules', len(filtered_df))

        # Display TA/Indication-level averages
        display(ta_averages)

else:
    print("Invalid input. Please enter 'M' for Molecule or 'T' for Therapeutic Area/Indication.")


Do you want to search by Molecule Name (M) or Other (Therapeutic Area/Indication) (O)? Enter 'M' or 'O':  O
Enter Therapeutic Area or Indication:  Oncology


,Search Term,Number of Molecules,Avg_Molecule_COGS,Avg_Molecule_SGA,Avg_Molecule_Marketing,Avg_Molecule_COGS_%_Sales,Avg_Molecule_SGA_%_Sales,Avg_Molecule_Marketing_%_Sales
0,Oncology,10,3079735973.904803,1513203262.900406,887761422.174385,25.205881,12.509199,7.293467
